In [ ]:
import os
import hashlib
import tkinter as tk
from tkinter import filedialog
from tkinter import simpledialog

# Liste der zu überprüfenden Dateiendungen (Syntax. ".txt", ".xlsx" usw)
# Hinweis: Wenn die Liste leer ist, dann werden alle Dateitypen überprüft
list_file_extensions = []

def select_mode():
    root = tk.Tk()
    root.withdraw()  # Verstecke das Hauptfenster

    # Zeige ein Dialogfenster mit den Optionen
    choice = simpledialog.askinteger("Auswahl", "1) Duplikatermittlung anhand von Dateinamen\n 2) Duplikatermittlung anhand Dateihashes\nBitte eine Option wählen (1 oder 2)", minvalue=1, maxvalue=2)

    if choice == 1:
        delete_duplicate_files_names(source_dir, target_dir)
    elif choice == 2:
        delete_duplicate_files_hashes(source_dir, target_dir)
    root.mainloop()

def get_all_files(directory):
    for root, dirs, files in os.walk(directory):
        for file in files:
            if not list_file_extensions or os.path.splitext(file)[1] in list_file_extensions:
                yield os.path.join(root, file)

def get_file_hash(file_path):
    with open(file_path, "rb") as f:
        bytes = f.read()
        readable_hash = hashlib.sha256(bytes).hexdigest()
    return readable_hash

def delete_duplicate_files_names(source_dir, target_dir):
    target_files = {os.path.basename(file) for file in get_all_files(target_dir)}
    removed_files = []

    for file in get_all_files(source_dir):
        if os.path.basename(file) in target_files:
            os.remove(file)
            print(f"Datei entfernt: {file}")
            removed_files.append(file)

    with open(os.path.join(source_dir, "entfernte Duplikate.txt"), "w") as f:
        for file in removed_files:
            f.write(f"{file}\n")

def delete_duplicate_files_hashes(source_dir, target_dir):
    target_files = {get_file_hash(file): file for file in get_all_files(target_dir)}

    for file in get_all_files(source_dir):
        if get_file_hash(file) in target_files:
            os.remove(file)
            print(f"Datei entfernt: {file}")

root = tk.Tk()
root.withdraw()

# Verzeichnisse hier angeben
source_dir = filedialog.askdirectory(title="Auswahl Quellverzeichnis (aus diesem werden die Duplikate entfernt)")
target_dir = filedialog.askdirectory(title="Auswahl Zielverzeichnis")

if __name__ == "__main__":
    select_mode()